In [20]:
from pydantic import BaseModel

class User(BaseModel):
    name: bool
    age: int
    email: str

user = User(name="0", age="30", email="")
print(user)


name=False age=30 email=''


In [58]:
from pydantic import BaseModel, condecimal
from decimal import Decimal

class Product(BaseModel):
    name: str
    price: condecimal(max_digits=4, decimal_places=3)  # max_digits and decimal_places for decimals

print(Product(name="foo", price=Decimal('7.995')))


name='foo' price=Decimal('7.995')


In [64]:
from pydantic import BaseModel, field_validator

class Person(BaseModel):
    name: str
    age: int

    @field_validator('age')
    def check_age(cls, value):
        if value < 18:
            raise ValueError('Must be at least 18 years old')
        return value

print(Person(name="akashvani",age=18))

name='akashvani' age=18


In [71]:
from pydantic import BaseModel

class ChatRequest(BaseModel):
    prompt: str
    max_tokens: int = Field(..., gt=0)  # Ensure positive token count

class ChatResponse(BaseModel):
    generated_text: str
    token_usage: int = Field(..., ge=0)

# Validate a request and response
request = ChatRequest(prompt="What is the weather?", max_tokens=50)
response = ChatResponse(generated_text="It is sunny today.", token_usage=45)



50

In [82]:
getattr(ChatRequest(prompt="What is the weather?", max_tokens=50), "prompt")

'What is the weather?'

In [73]:
from typing import Optional
from pydantic_settings import BaseSettings
from functools import lru_cache
from dotenv import load_dotenv
import os

load_dotenv()


class LLMProviderSettings(BaseSettings):
    temperature: float = 0.0
    max_tokens: Optional[int] = None
    max_retries: int = 3


class OpenAISettings(LLMProviderSettings):
    # api_key: str = os.getenv("OPENAI_API_KEY")
    default_model: str = "gpt-4o"


class AnthropicSettings(LLMProviderSettings):
    # api_key: str = os.getenv("ANTHROPIC_API_KEY")
    default_model: str = "claude-3-5-sonnet-20240620"
    max_tokens: int = 1024


class LlamaSettings(LLMProviderSettings):
    api_key: str = "key"  # required, but not used
    default_model: str = "llama3"
    base_url: str = "http://localhost:11434/v1"


class Settings(BaseSettings):
    app_name: str = "GenAI Project Template"
    openai: OpenAISettings = OpenAISettings()
    anthropic: AnthropicSettings = AnthropicSettings()
    llama: LlamaSettings = LlamaSettings()


@lru_cache
def get_settings():
    return Settings()